## Silver: Comercio

In [0]:
%run "../1 - Setup/Ingestao_bibliotecas_padrao"


In [0]:
from pyspark.sql.functions import col

In [0]:

df_original = spark.table("vitivinicultura.bronze_comercializacao.comercio").toPandas()

#  Define a coluna que contém os dados separados por ";"
coluna = "_c0"

#  colunas adicionais 
colunas_adicionais = ["ingestao_ts", "usuario_ingestao"]  #
df_extra = df_original[colunas_adicionais].copy()

#  Quebra a string em colunas separadas
linhas = df_original[coluna].str.split(';', expand=True)

#  Usa a primeira linha como cabeçalho
linhas.columns = linhas.iloc[0]

#  Remove a primeira linha (que era o cabeçalho)
df_tratado = linhas[1:].reset_index(drop=True)

#  Junta as colunas adicionais ao dataframe tratado
df_final = pd.concat([df_extra.reset_index(drop=True), df_tratado], axis=1)

# Visualiza o resultado final
display(df_final)

In [0]:
 #Cria tabela gerenciada no Unity Catalog
spark.sql("""
CREATE TABLE IF NOT EXISTS vitivinicultura.silver_comercializacao.comercio
USING DELTA
""")
df_spark = spark.createDataFrame(df_final)
# Escrita no formato Delta Lake
df_spark.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("vitivinicultura.silver_comercializacao.comercio")

